# Distilbert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers==4.30

In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_2_balanced.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Distilbert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.973918,0.072917,0.062809,0.067143,0.030319
2,No log,2.741964,0.375000,0.436533,0.411629,0.332619
3,No log,2.236989,0.520833,0.641896,0.578003,0.514008
4,No log,1.870791,0.520833,0.639338,0.554129,0.491603
5,No log,1.459246,0.593750,0.671389,0.629594,0.576436
6,No log,1.263516,0.614583,0.610491,0.662468,0.585918
7,No log,0.877371,0.718750,0.795929,0.742435,0.730821
8,No log,0.868187,0.760417,0.782381,0.771623,0.748896
9,No log,0.957969,0.750000,0.780114,0.772911,0.755364
10,1.374600,0.846574,0.791667,0.825808,0.812078,0.796456


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=648, training_loss=1.0645496370615783, metrics={'train_runtime': 245.7524, 'train_samples_per_second': 42.091, 'train_steps_per_second': 2.637, 'total_flos': 1370682625720320.0, 'train_loss': 1.0645496370615783, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-0.84277344,  0.56884766,  6.3710938 , ..., -0.9291992 ,
        -0.14953613, -1.5117188 ],
       [-1.7275391 , -0.45825195, -2.5429688 , ..., -1.9306641 ,
        -2.2246094 , -1.5507812 ],
       [-2.015625  , -2.0019531 , -0.9482422 , ..., -2.03125   ,
         0.05548096, -0.93896484],
       ...,
       [ 0.02415466, -0.29833984, -2.1152344 , ..., -1.7451172 ,
        -1.8203125 , -1.8916016 ],
       [-0.73291016,  1.0253906 ,  6.75      , ..., -0.09466553,
        -0.4321289 , -1.4804688 ],
       [-1.4179688 , -0.95166016, -2.1796875 , ..., -1.5947266 ,
        -1.4638672 , -0.4765625 ]], dtype=float32), label_ids=array([ 2,  5, 16,  3,  4,  1, 19, 17, 13, 11,  1,  4,  4, 11,  9, 10, 12,
        8,  1,  3, 13,  7,  6, 19, 16,  9,  6,  0, 19,  7, 13, 14,  6,  8,
       13,  0,  3, 16, 19,  7, 17, 16,  5, 19, 11, 19, 19,  4,  4,  9,  8,
       14, 13, 10, 11, 19,  5,  5, 18, 13, 19, 10, 15, 16, 10,  8,  4,  3,
        0, 12,  3, 10,  2,  4, 1

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.966878,0.104167,0.120121,0.176786,0.125813
2,No log,2.646373,0.302083,0.227771,0.346748,0.222729
3,No log,1.973602,0.510417,0.505883,0.523945,0.454318
4,No log,1.511313,0.604167,0.602376,0.624459,0.574697
5,No log,1.134670,0.656250,0.691012,0.676921,0.659703
6,No log,0.910051,0.729167,0.719444,0.726623,0.708058
7,No log,0.861365,0.750000,0.787143,0.753766,0.737006
8,No log,1.031184,0.802083,0.842513,0.811364,0.796319
9,No log,0.943246,0.781250,0.809563,0.798387,0.768506
10,1.199800,0.880375,0.791667,0.805833,0.804221,0.785051


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=648, training_loss=0.9271175313143083, metrics={'train_runtime': 248.6343, 'train_samples_per_second': 41.603, 'train_steps_per_second': 2.606, 'total_flos': 1370682625720320.0, 'train_loss': 0.9271175313143083, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-0.42822266,  0.19726562,  6.421875  , ..., -1.2675781 ,
        -1.3095703 , -1.5810547 ],
       [ 0.03677368,  3.5019531 , -0.43310547, ..., -2.7675781 ,
        -1.0537109 , -2.6855469 ],
       [-1.9404297 , -0.47094727,  0.19726562, ..., -1.3076172 ,
         0.06213379, -2.0546875 ],
       ...,
       [ 5.4765625 ,  0.06677246, -1.6699219 , ..., -1.6650391 ,
        -2.5683594 , -2.8027344 ],
       [-0.3190918 ,  0.6513672 ,  6.5273438 , ..., -0.7998047 ,
        -1.7724609 , -1.6708984 ],
       [-1.7675781 , -1.5449219 , -1.3105469 , ..., -2.1738281 ,
        -1.3828125 , -0.6586914 ]], dtype=float32), label_ids=array([ 2,  5, 16,  3,  4,  1, 19, 17, 13, 11,  1,  4,  4, 11,  9, 10, 12,
        8,  1,  3, 13,  7,  6, 19, 16,  9,  6,  0, 19,  7, 13, 14,  6,  8,
       13,  0,  3, 16, 19,  7, 17, 16,  5, 19, 11, 19, 19,  4,  4,  9,  8,
       14, 13, 10, 11, 19,  5,  5, 18, 13, 19, 10, 15, 16, 10,  8,  4,  3,
        0, 12,  3, 10,  2,  4, 1